In [ ]:
data_loc = ""
label_loc = ""
ckpt_loc = ""


In [ ]:
from transGNN.Data.paras import ReaderParas
reader_para = ReaderParas()
reader_para.data_file_loc = data_loc
reader_para.data_pid = 'PatientID'
reader_para.label_file_loc = label_loc
reader_para.label_pid = 'PatientID'
reader_para.label_name = 'HRD'
reader_para.label_dict = {'HRD':1, 'HR-proficient':0}
reader_para.with_transpose = False
reader_para.gene_thresh = None
reader_para.adj_thresh = 0.1

reader_para.test_ratio = 0.01
reader_para.batch_size = 1

In [ ]:
from transGNN.Data.dataset import create_dataset
#train_loader,test_loader,trainset,testset,reader,class_weights = create_dataset(reader_para)
import torch
from transGNN.Data.paras import ReaderParas
from transGNN.Data.preprocessing import DataReader
from torch.utils.data import random_split,TensorDataset,DataLoader

reader = DataReader(reader_para) 
features_table, labels_str = reader.data_readfiles()
reader.get_adj_M() # get adj matrix

full_dataset = TensorDataset(features_table, labels_str)

test_size = int(reader_para.test_ratio * len(full_dataset))
train_size = len(full_dataset) - test_size

# split dataset with random or K-fold
trainset,testset = random_split(full_dataset,
                lengths=[train_size,test_size],
                generator=torch.Generator().manual_seed(0)
                )

In [ ]:

# random weighted sampler 
# sampler

batch_size = reader_para.batch_size


from torch.utils.data.sampler import WeightedRandomSampler
# 
# 
num_classes = len(reader_para.label_dict.keys())
# 
class_counts = [0] * num_classes  #
for _, label in trainset:
    class_counts[int(label)] += 1

# 
class_weights = [len(trainset) / class_counts[i] for i in range(num_classes)]

# 
weights = [class_weights[int(label)] for _, label in trainset]

trainsampler = WeightedRandomSampler(weights, len(weights))

#weights = [1.0 / len(testset) for i in range(len(testset))]
#testsampler = WeightedRandomSampler(weights, len(weights))

# 
train_loader = DataLoader(dataset=trainset, 
                                batch_size=batch_size,sampler=trainsampler) #shuffle=True)
test_loader = DataLoader(dataset=testset, 
                                batch_size=batch_size, shuffle=False)



In [ ]:
print(reader.paras.label_dict,class_counts)
print(class_weights)

In [ ]:
from transGNN.Model.GAT.paras import GATParas
from transGNN.Predefine.GAT_classifier import gat_model_trainer_para
from transGNN.Model.GAT.pl import pl_GAT
gat_model_trainer_para.project = "DNAD_GNN_GAT" # project name
gat_model_trainer_para.entity= "shipan_work" # entity name
gat_model_trainer_para.exp_name = "GNN_GAT" # experiment name
gat_model_trainer_para.class_nb = len(reader.paras.label_dict.keys())
gat_model_trainer_para.batch_size = reader_para.batch_size

gat_model_trainer_para.optimizers:list = [torch.optim.Adam,
                       ] # list of optimizer
gat_model_trainer_para.optimizer_paras:list =[
        {"lr":1e-4},# for first optimizer
    ] # list of optimizer paras dict
"""
gat_model_trainer_para.schedulers:list = [torch.optim.lr_scheduler.StepLR] # list of scheduler
gat_model_trainer_para.scheduler_paras:list = [{
                            #"decay_step":10,
                            "step_size":800,
                            "gamma":0.5},] # list of scheduler paras dict

"""


gat_model_trainer_para.save_ckpt = True # save checkpoint or not
gat_model_trainer_para.ckpt_folder = "/Users/shipan/Documents/workspace_transGNN/pretrained/"
#debug: try to add formated name to ckpt
gat_model_trainer_para.ckpt_format:str = "_{epoch:02d}-{auroc_train:.2f}" # check_point format 
gat_model_trainer_para.ckpt_para = { #-----------> paras for pytorch_lightning.callbacks.ModelCheckpoint
                    "save_top_k":1,
                   "monitor":"auroc_train",
                   "mode":"max",}
                   
gat_model_trainer_para.with_logger = "wandb" # "wandb",
gat_model_trainer_para.wandb_api_key = "Your wandb api key"

gat_model_trainer_para.max_epochs = 40 # max epochs

model_paras = GATParas()
model_paras.input_dim = 1
model_paras.lin_input_dim=681
model_paras.is_fixed_adj_matrix = True
pl_model = pl_GAT(trainer_paras=gat_model_trainer_para,model_paras=model_paras)


In [ ]:
pl_model.create_model()
pl_model.set_adj_M(reader.M)

pl_model.build_trainer()

In [ ]:
len(train_loader)

In [ ]:
pl_model.loss_name

In [ ]:
#pl_model.trainer.fit(pl_model,train_loader,test_loader)

In [ ]:
# load checkpoint for pytorch lightning model
import torch
pl_model.load_state_dict(torch.load(ckpt_loc,map_location=torch.device('cpu'))["state_dict"])


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F

########################################################################
# .    model requires gradient calculation functions for XAI
########################################################################
def calc_GAT_gradient(data,model,class_nb=2,concat_nb=3):
    """
    in:
        data: pyg_tensor with x and y
        model: pytorch model
    
    out:
        gradient: saved gradient for y_c = torch.sum(one_hot_labels*out)
        feature_importance: feature importance for current data 
    """
    grad_labels = data.y.to(torch.int64)

    result_dict=model(data)

    GAT_features = result_dict["GAT_features"]
    out = result_dict["logits"]

    # clean gradient
    GAT_features.grad = None
    GAT_features.retain_grad()
    
    # to one hot
    one_hot_labels = F.one_hot(grad_labels, num_classes=class_nb)
    one_hot_labels = one_hot_labels.to(GAT_features.device)
    y_c = torch.sum(one_hot_labels*out)

    y_c.backward(retain_graph=True)
    np_GAT_f_grad = GAT_features.grad.detach().cpu().numpy()
    np_GAT_f_grad = np_GAT_f_grad.reshape([1,int(np_GAT_f_grad.shape[0]/concat_nb),concat_nb])
    #print(np_GAT_f_grad.shape)
    gradients = np.maximum(np_GAT_f_grad, 0)# (1, feature_size,concat_nb)
    feature_importance = np.mean(gradients, 0)

    return gradients,feature_importance

def class_importance(target_class,labels,gradients,concat_nb,feature_dim):
    """
    in:
        target_class:int: target class for importance
        labels:np.ndarray: a list of number as labels
        gradients:
    return:
        class_i_importance:np.ndarray: importance score for class i
    """
    class_i_index = np.argwhere(labels == target_class)
    class_i_gradients = gradients[class_i_index[:, 0], :]
    class_i_importance = np.mean(class_i_gradients, axis=0)
    class_i_importance = np.reshape(class_i_importance, (concat_nb, feature_dim)).T
    from sklearn.preprocessing import normalize
    #print(class_i_importance.shape)
    norm_class_i_importance = normalize(class_i_importance, axis=0, norm='max')
    #print(norm_class_i_importance.shape)
    class_i_importance = np.expand_dims(norm_class_i_importance, axis=0)

    return class_i_importance #[1, feature_dim, concat_nb]


In [ ]:
def infer_importance(dataloader,pl_model,reader):
    gradient_matrixs=[]
    labels = []
    class_nb = len(reader.paras.label_dict.keys())
    concat_nb = 3 
    for _,(x,y) in enumerate(dataloader):
        data = pl_model.get_graph_data(pl_model.M,x,y)
        # not pl_model need pytorch model forward
        gradients,_=calc_GAT_gradient(data,pl_model.model,
                                        class_nb=class_nb,concat_nb=concat_nb) #G = [1,feat_dim,cat_nb]
        gradient_matrixs.append(gradients)
        labels.append(y)
    importance_list = []
    for i in range(class_nb):
        imp_i = class_importance(target_class=i,
                        labels=np.array(labels),
                        gradients=np.array(gradient_matrixs),
                        concat_nb=concat_nb,
                        feature_dim=gradient_matrixs[0].shape[-2])
        importance_list.append(imp_i)

    np_imps = np.concatenate(importance_list,axis=0)
    #layer_avg=np.mean(np_imps, axis=-1, keepdims=True)
    layer_avg = np.sum(np_imps, axis=-1, keepdims=True)
    def normalization(data):
        _range = np.max(data) - np.min(data)
        return (data - np.min(data)) / _range
    layer_avg[0,...] = normalization(layer_avg[0,...])
    layer_avg[1,...] = normalization(layer_avg[1,...])
    np_imps_all = np.concatenate((np_imps, layer_avg), axis=-1)# [class_nb,feat_dim,concat_nb+1]
    return np_imps_all


In [ ]:
np_imps_all = infer_importance(train_loader,pl_model,reader)

In [ ]:
#[class_nb,feat_dim,concat_nb+1]
np_imps_all.shape

In [ ]:
reader.paras.label_dict

In [ ]:
# dataframe 
gene_names = reader.df_data.columns.tolist()[1:]
#{'HRD': 1, 'HR-proficient': 0}
#normalise to 0-1
d_0 = np_imps_all[0,:,-1]
d_0=d_0-d_0.min()/d_0.max()-d_0.min()

d_1 = np_imps_all[1,:,-1]
d_1=d_1-d_1.min()/d_1.max()-d_1.min()

# create dataframe
data_dict = {
    'gene_names': gene_names,
    "HR-proficient":d_0, #normalise to 0-1
    "HRD":d_1,
}
import pandas as pd
df = pd.DataFrame(data_dict)

In [ ]:
df.to_csv("imp_score_avg.csv")